# Re-extract subject headings 

In [2]:
import sys, re, os 
from bs4 import BeautifulSoup, SoupStrainer
from tqdm import tqdm  
import pandas as pd 
TCP = '/Users/amycweng/DH/TCP'
sys.path.append('../')

def findTextTCP(id):
    if re.match('B1|B4',id[0:2]):
        path = f'{TCP}/P2{id[0:2]}/{id}.P4.xml'
    else: 
        if f'{id}.P4.xml' in os.listdir(f'{TCP}/P1{id[0:2]}'):
            path = f'{TCP}/P1{id[0:2]}/{id}.P4.xml'
        elif f'{id}.P4.xml' in os.listdir(f'{TCP}/P2{id[0:2]}'): 
            path = f'{TCP}/P2{id[0:2]}/{id}.P4.xml'
    return path 

In [3]:
sermons = list(pd.read_csv("../assets/sermons.csv")['id'])
sermons.extend(list(pd.read_csv("../assets/sermons_missing.csv")["id"]))
tcpID_subjects = {}
progress_bar = tqdm(sermons)
for file in progress_bar:
    progress_bar.set_description(file)
    tcpID = file.split(".")[0]
    tcpID_subjects[tcpID] = []
    filepath = findTextTCP(tcpID)

    # read the input XML file 
    with open(filepath,'r',encoding='utf-8') as file: 
        data = file.read()
    # use soupstrainer to only parse the main body
    tag = SoupStrainer("KEYWORDS")
    soup = BeautifulSoup(data,features="xml",parse_only=tag)
    
    subjects = soup.find_all(['TERM'])
    for s in subjects: 
        tcpID_subjects[tcpID].append(s)

A27574: 100%|██████████| 5729/5729 [04:13<00:00, 22.60it/s]


In [4]:
tcpID_subjects = {k:[item.string for item in v] for k,v in tcpID_subjects.items()}

In [5]:
from tqdm import tqdm 
def correct_metadata_file(fp):
    sermons = pd.read_csv(fp)
    sermons = sermons.to_dict(orient='records')
    new_sermons = []
    for info_dict in tqdm(sermons):
        tcpID = info_dict['id']
        filepath = findTextTCP(tcpID)
        if re.search(f'TCP/P1',filepath): 
            phase = "1"
        else: 
            phase = "2"
        info_dict['phase'] = phase
        new_sermons.append(info_dict)
    new_sermons = pd.DataFrame(new_sermons)
    new_sermons.to_csv(fp,index=False)

In [6]:
correct_metadata_file("../assets/sermons.csv")

100%|██████████| 4226/4226 [00:29<00:00, 142.88it/s]


In [7]:
correct_metadata_file("../assets/sermons_missing.csv")


100%|██████████| 1503/1503 [00:08<00:00, 169.77it/s]


In [9]:
import pandas as pd 
from collections import Counter 
from tqdm import tqdm 
import re 

sermons = pd.read_csv("/Users/amycweng/DH/SERMONS_APP/db/data/sermons.csv",header=None)
sermons = sermons.to_dict(orient='records')

missing = pd.read_csv("/Users/amycweng/DH/SERMONS_APP/db/data/sermons_missing.csv",header=None)
missing = missing.to_dict(orient='records')
sermons.extend(missing)

export = []
all_subjects = []

for info_dict in tqdm(sermons):
    subjects = info_dict[7]
    if not isinstance(subjects,str): continue  
    seen = {}
    for w in subjects.split("; "): 
        if w == "No Keywords": continue
        if w != 'O.T.' and w!= 'N.T.': 
            w = w.strip(".")
        # w = re.sub(r"\s+"," ",w)
        # w = re.sub(" -- Early works to 1800","",w)
        # w = re.sub(r"\s+"," ",w)
        # w = re.sub(" -- 16th century| -- 17th century","",w)
        # w = re.sub(r"\s+"," ",w)
        # w = re.sub(" -- Sermons","",w)
        # w = re.sub(r"\s+"," ",w)
        # w = re.sub("Great Britain -- History -- ","",w)
        # w = re.sub(r"\s+"," ",w)
        all_subjects.append(w)
        if w not in seen: 
            seen[w] = True 
            export.append({'tcpID':info_dict[0],'subject':w})

processed = {}
for w, freq in sorted(dict(Counter(all_subjects)).items(),key=lambda x:x[1],reverse=True):
    if w not in processed: processed[w] = freq
    else: processed[w] += freq
import json,csv 
outputfolder = "/Users/amycweng/DH/SERMONS_APP/db/data"

with open(f"{outputfolder}/subjects.json","w+") as file: 
    json.dump(sorted(processed.items(),key=lambda x:x[1],reverse=True),file)
with open(f"{outputfolder}/subjects.csv","w+") as outfile: 
    writer = csv.DictWriter(outfile, fieldnames=export[0].keys())
    writer.writerows(export)

100%|██████████| 5729/5729 [00:00<00:00, 201419.69it/s]
